In [17]:
import pandas as pd
from IPython.display import display

from connection_configuration import get_secret_from_keyvault, create_connection
from sqlalchemy.types import Integer, String

## Create a connection
the first example is a simple connection to a sql server. 

In [18]:
user = 'sqladm'
password = get_secret_from_keyvault('moneyhubkeyvault', 'sqladm')
server = 'moneyhubsqlserver.database.windows.net'
database = 'moneyhubsqldev'
engine, connection = create_connection(user,password,server,database)

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,moneyhubsqldev,sys,database_firewall_rules,VIEW
1,moneyhubsqldev,DW,DimAccount,BASE TABLE
2,moneyhubsqldev,DW,DimDate,BASE TABLE
3,moneyhubsqldev,DW,DimTransaction,BASE TABLE
4,moneyhubsqldev,DW,FactTransaction,BASE TABLE
5,moneyhubsqldev,DW,Sample,BASE TABLE


note that in order to make the keyvault work you need the following steps to setup in powershell:
```
Connect-AzAccount
Set-AzKeyVaultAccessPolicy -VaultName "<your-unique-keyvault-name>" -UserPrincipalName "user@domain.com" -PermissionsToSecrets delete,get,list,set
```

the set policy is only applicable if not already done in azure portal. 
Connect-AzAccount is to force the correct service principle to be used

# read from a table

In [19]:
sql = """SELECT * FROM DW.FactTransaction;"""

pd.read_sql(sql, connection)

,DateKey,AccountKey,TransactionKey,Description,Amount
0,20220525,3,52,TIM HORTONS #3923 ABBOTSFORD BC,-4.0
1,20220725,2,100,Internet Banking INTERNET TRANSFER 000000229517,2200.0
2,20220701,4,52,HAPPY BAGEL BAKERY TORONTO ON (GOO...,-12.0
3,20220725,3,64,PURCHASE INTEREST CHARGE,0.0
4,20220522,3,52,VALUE VILLAGE # 2140 MISSION BC,-34.0
...,...,...,...,...,...
1263,20210726,1,47,Internet Banking INTERNET TRANSFER 000000272605,-18.0
1264,20210726,1,89,Internet Banking INTERNET BILL PAY 00000026417...,-631.0
1265,20210719,1,90,Internet Banking E-TRANSFER 010355312363 SINA ...,1000.0
1266,20210719,1,90,Internet Banking E-TRANSFER 010354945225 SINA ...,255.0


# write to a table
Note that for writing tables you can use sql alchemies database specific types. See the docs for UPPERCASE types

In [24]:
sample = pd.read_excel('SampleInput.xlsx',0)
display(sample)

types = {'Name':String,'LastName':String,'Badge':Integer}

sample.to_sql(name='Sample',con=connection,schema='DW',if_exists='replace',index=False,dtype=types)

sql = """SELECT * FROM DW.Sample;"""

display(pd.read_sql(sql, connection))

sql = """
select schema_name(tab.schema_id) as schema_name,
    tab.name as table_name, 
    col.column_id,
    col.name as column_name, 
    t.name as data_type,    
    col.max_length,
    col.precision
from sys.tables as tab
    inner join sys.columns as col
        on tab.object_id = col.object_id
    left join sys.types as t
    on col.user_type_id = t.user_type_id
where schema_name(tab.schema_id) = 'DW'
and tab.name = 'Sample';
"""
display(pd.read_sql(sql, connection))



,Name,LastName,Badge
0,John,Doe,198.0
1,Sam,Willson,NaN
2,Terry,Cruze,823.0


,Name,LastName,Badge
0,John,Doe,198.0
1,Sam,Willson,NaN
2,Terry,Cruze,823.0


,schema_name,table_name,column_id,column_name,data_type,max_length,precision
0,DW,Sample,1,Name,varchar,-1,0
1,DW,Sample,2,LastName,varchar,-1,0
2,DW,Sample,3,Badge,int,4,10
